In [1]:
# ! gdown --id 1GY5z0Op33YGE28ImNf9ycrEs1cDqXDV6

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1GY5z0Op33YGE28ImNf9ycrEs1cDqXDV6
To: /content/users.csv
100% 29.9M/29.9M [00:00<00:00, 165MB/s] 


In [2]:
# ! gdown --id 1Njfi_ITa6rMYOu_HUXQXkGnNiRwn7Wic

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Njfi_ITa6rMYOu_HUXQXkGnNiRwn7Wic
To: /content/interactions.csv
100% 180M/180M [00:01<00:00, 126MB/s]


In [3]:
# ! gdown --id 1SKBJ6JJPV9R-kivP8xCDLaEjoPPdwrUc

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1SKBJ6JJPV9R-kivP8xCDLaEjoPPdwrUc
To: /content/items.csv
100% 31.8M/31.8M [00:00<00:00, 123MB/s]


In [4]:
# %pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.6 MB/s eta 0:00:00


In [5]:
# %pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808326 sha256=a25411ccc8ba483f3832474715376483e58d2335a50f76d59cca6107ef3f348e
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [6]:
# %pip install rectools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 28.1 MB/s eta 0:00:00


In [7]:
# !pip install implicit

In [8]:
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import PopularModel, RandomModel, ImplicitALSWrapperModel
from rectools import Columns
from rectools.model_selection import TimeRangeSplitter
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
from pathlib import Path
import typing as tp
from tqdm import tqdm

from lightfm import LightFM

from implicit.bpr import BayesianPersonalizedRanking

from implicit.lmf import LogisticMatrixFactorization

import optuna
from optuna.samplers import TPESampler

In [9]:
interactions = pd.read_csv("/content/interactions.csv")
users = pd.read_csv("/content/users.csv")
items = pd.read_csv("/content/items.csv")

In [10]:
Columns.Datetime = 'last_watch_dt'

In [11]:
def display_df(df):
    return pd.concat([df.head(), df.tail()])

In [12]:
display_df(interactions)

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


### Users

In [13]:
users.isnull().sum()

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64

In [14]:
users.dtypes

user_id      int64
age         object
income      object
sex         object
kids_flg     int64
dtype: object

In [15]:
users.fillna("Unknown", inplace=True)

In [16]:
users.nunique()

user_id     840197
age              7
income           7
sex              3
kids_flg         2
dtype: int64

In [17]:
users.isnull().sum()

user_id     0
age         0
income      0
sex         0
kids_flg    0
dtype: int64

In [18]:
features = {
    "users": ["age", "income", "sex"]
}

### Items


In [19]:
display_df(items)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
15958,6443,series,Полярный круг,Arctic Circle,2018.0,"драмы, триллеры, криминал","Финляндия, Германия",NaN,16.0,NaN,Ханну Салонен,"Иина Куустонен, Максимилиан Брюкнер, Пихла Вии...","Во время погони за браконьерами по лесу, сотру...","убийство, вирус, расследование преступления, н..."
15959,2367,series,Надежда,NaN,2020.0,"драмы, боевики",Россия,0.0,18.0,NaN,Елена Хазанова,"Виктория Исакова, Александр Кузьмин, Алексей М...",Оригинальный киносериал от создателей «Бывших»...,"Надежда, 2020, Россия"
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Р...",Криминальная драма по мотивам романов о шведск...,"Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родд...",Семнадцатилетний Дэмиен мечтает вырваться за п...,"Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы...","Гоша, 2019, Россия"


In [20]:
items.isnull().sum()

item_id             0
content_type        0
title               0
title_orig       4745
release_year       98
genres              0
countries          37
for_kids        15397
age_rating          2
studios         14898
directors        1509
actors           2619
description         2
keywords          423
dtype: int64

In [21]:
items.nunique()

item_id         15963
content_type        2
title           15293
title_orig      10575
release_year      105
genres           2771
countries         687
for_kids            2
age_rating          6
studios            38
directors        7973
actors          12996
description     15619
keywords        15492
dtype: int64

In [22]:
items["countries"].fillna("Unknown", inplace=True)

In [23]:
items["genres"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
items["countries"] = items["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")

In [24]:
features.update(
    {
        "items": ["content_type", "genres", "countries"]
    }
)

**Фичи**, которые **будут использоваться** для users-ов и item-ов соответственно


In [25]:
features

{'users': ['age', 'income', 'sex'],
 'items': ['content_type', 'genres', 'countries']}

In [26]:
Columns.Datetime = 'last_watch_dt'

In [27]:
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


In [28]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')

In [29]:
max_date = interactions[Columns.Datetime].max()
max_date

Timestamp('2021-08-22 00:00:00')

In [30]:
# Бинаризация процента просмотренного на 10 частей
interactions[Columns.Weight] = pd.qcut(
    interactions["watched_pct"], 10, duplicates = "drop"
).astype("category").cat.codes

### Разделяем по фолдам по 7 дней

In [31]:
# Number of splits for time-based cross-validation
N_SPLITS = 3

# Time duration for each test set in the cross-validation
TEST_SIZE = '7D'  # 7 days

# Initializing a generator for time-based cross-validation folds
cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [32]:
from rectools.dataset import Interactions
len(cv.get_test_fold_borders(Interactions(interactions)))

3

### Метрики


In [33]:
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "MAP@10": MAP(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
}

metrics

{'prec@10': Precision(k=10),
 'recall@10': Recall(k=10),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'novelty': MeanInvUserFreq(k=10),
 'serendipity': Serendipity(k=10)}

### Модели


In [34]:
suggested_params = {}

# создаем набор данных с юзерами и их характеристками

def create_features_dataset(col, train, df_value, features):
    df_value = df_value.loc[df_value[col].isin(train[col])].copy()

    features_frames = []
    for feature in features:
        feature_frame = df_value.reindex(columns=[col, feature])
        if feature in ["genres", "countries"]:
            feature_frame = feature_frame.explode(feature)
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature
        features_frames.append(feature_frame)

    return pd.concat(features_frames)

In [35]:
def cross_val(model, cv, interactions_df, user_df, item_df, features, k_recos):

    results = list()
    # Создание итератора для перекрестной проверки (cross-validation)
    fold_iterator = cv.split(Interactions(interactions), collect_fold_stats=True)

    # Перебор каждого набора данных для обучения и тестирования
    for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
        print(f"\n==================== Fold {i_fold}")

        # Получение обучающего и тестового набора данных из взаимодействий
        train = interactions.iloc[train_ids].copy()
        test = interactions.iloc[test_ids][Columns.UserItem].copy()

        # удаляем те строки, где просмотр менее 300 секунд
        train.drop(train.query("total_dur < 300").index, inplace=True)
        # выдялем cold users
        cold_users = set(test[Columns.User]) - set(train[Columns.User])
        # удаляем из теста холодным юзеров
        test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

         # Создание набора данных характеристик для пользователей
        user_features = create_features_dataset(Columns.User, train, user_df, features["users"])
        # Создание набора данных характеристик для айтема
        item_features = create_features_dataset(Columns.Item, train, item_df, features["items"])

        # Cоздаем датасет
        dataset = Dataset.construct(
            interactions_df=train,
            user_features_df=user_features,
            cat_user_features=features["users"],
            item_features_df=item_features,
            cat_item_features=features["items"]
        )

        # Получение уникальных пользователей и элементов для тестирования
        user_test = test[Columns.User].unique()
        catalog = train[Columns.Item].unique()

        # Обучение модели на подготовленном наборе данных
        model.fit(dataset)

        # Получение рекомендаций модели для тестовых пользователей
        recos = model.recommend(
            users=user_test,
            dataset=dataset,
            k=k_recos,
            filter_viewed=True,
        )

        # Вычисление метрик для оценки качества рекомендаций
        metric_values = calc_metrics(metrics, recos, test, train, catalog)
        results.append(metric_values)

    # Создание DataFrame из результатов и вычисление средних значений по метрикам
    df_result = pd.DataFrame(results)

    # Возвращение средних значений метрик
    return df_result.mean(axis=0).tolist()

In [36]:
# функция для подбора гиперпараметров для модели ALS
def objective_als(trial, config, cv, interactions_df, user_df, item_df, features, metrics):

    MODEL = "ALS"

    n_factors = trial.suggest_categorical("n_factors", config["N_FACTORS"])

    is_fitting_features = trial.suggest_categorical(
        "is_fitting_features",
        config["is_fitting_features"]
    )

    if f"{MODEL}_{n_factors}_{is_fitting_features}" in suggested_params:
        suggested_params[f"{MODEL}_{n_factors}_{is_fitting_features}"] += 1
        raise optuna.exceptions.TrialPruned()
    else:
        suggested_params[f"{MODEL}_{n_factors}_{is_fitting_features}"] = 1

     # Создание модели ALS с выбранными параметрами
    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=n_factors,
            random_state=config["RANDOM_STATE"],
            num_threads=config["NUM_THREADS"],
        ),
        fit_features_together=is_fitting_features
    )

    trial.set_user_attr("n_factors", n_factors)
    trial.set_user_attr("is_fitting_features", is_fitting_features)

    print(f"***RUN***\nmodel: {MODEL}\nN_FACTORS: {n_factors}\nis_fitting_features: {is_fitting_features}\n")

    return cross_val(model, cv, interactions_df, user_df, item_df, features, k_recos=config["K_RECOS"])

In [37]:
# функция для подбора гиперпараметров для модели LightFM
def objective_lfm(trial, config, cv, interactions_df, user_df, item_df, features, metrics):

    MODEL = "LightFM"

    n_factors = trial.suggest_categorical("n_factors", config["N_FACTORS"])
    losses = trial.suggest_categorical("losses", config["lightfm_losses"])
    user_alpha = trial.suggest_categorical("user_alpha", config["USER_ALPHA"])
    item_alpha = trial.suggest_categorical("item_alpha", config["ITEM_ALPHA"])

    if f"{MODEL}_{n_factors}_{losses}_{user_alpha}_{item_alpha}" in suggested_params:
        suggested_params[f"{MODEL}_{n_factors}_{losses}_{user_alpha}_{item_alpha}"] += 1
        raise optuna.exceptions.TrialPruned()
    else:
        suggested_params[f"{MODEL}_{n_factors}_{losses}_{user_alpha}_{item_alpha}"] = 1

    model = LightFMWrapperModel(
        model=LightFM(
            no_components=n_factors,
            loss=losses,
            random_state=config["RANDOM_STATE"],
            learning_rate=config["LEARNING_RATE"],
            user_alpha=user_alpha,
            item_alpha=item_alpha,
        ),
        epochs=config["N_EPOCHS"],
        num_threads=config["NUM_THREADS"],
    )

    trial.set_user_attr("n_factors", n_factors)
    trial.set_user_attr("losses", losses)
    trial.set_user_attr("user_alpha", user_alpha)
    trial.set_user_attr("losses", item_alpha)

    print(f"***RUN***model: {MODEL}\nN_FACTORS: {n_factors}\nloss: {losses}\nuser_alpha: {user_alpha}\nitem_alpha: {item_alpha}\n")

    return cross_val(model, cv, interactions_df, user_df, item_df, features, k_recos=config["K_RECOS"])

In [38]:
CONFIG_MODELS = {
    "K_RECOS": 10,
    "RANDOM_STATE": 42,
    "NUM_THREADS": 16,
#     "N_FACTORS": (16, 32, ),
    "N_FACTORS": (4, ),
    "is_fitting_features": (False, True),  # ALS
    "N_EPOCHS": 10,  # Lightfm
    "USER_ALPHA": (0,),  # Lightfm
    "ITEM_ALPHA": (0,),  # Lightfm
    "LEARNING_RATE": 0.05,  # Lightfm
    "lightfm_losses": ('bpr', 'warp', ),  # Lightfm
}

In [39]:
%%time

# подбор гиперпараметров с помощью OPTUNA для модели ALS
study_als = optuna.create_study(
    study_name="ALS",
    sampler=TPESampler(seed=42),
    directions=['maximize', 'maximize', 'maximize', 'maximize', 'maximize']
)
# Запуск оптимизации гиперпараметров
study_als.optimize(
    lambda trial: objective_als(
        trial=trial,
        config=CONFIG_MODELS,
        cv=cv,
        interactions_df=interactions,
        user_df=users,
        item_df=items,
        features=features,
        metrics=metrics
    ),
    n_trials=int(
        len(CONFIG_MODELS["N_FACTORS"]) *
        len(CONFIG_MODELS["is_fitting_features"]) *
        2
    )
)

[I 2023-12-06 08:03:30,974] A new study created in memory with name: ALS


***RUN***
model: ALS
N_FACTORS: 4
is_fitting_features: True


==================== Fold 0


/usr/local/lib/python3.10/dist-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")



==================== Fold 1


/usr/local/lib/python3.10/dist-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


...

/usr/local/lib/python3.10/dist-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[I 2023-12-06 08:08:53,531] Trial 1 finished with values: [0.03182514947964032, 0.15525530016152608, 0.0754495184315937, 4.425381816592542, 1.7396060728361532e-05] and parameters: {'n_factors': 4, 'is_fitting_features': False}. 
[I 2023-12-06 08:08:53,534] Trial 2 pruned. 
[I 2023-12-06 08:08:53,537] Trial 3 pruned. 


CPU times: user 5min 44s, sys: 1min 19s, total: 7min 4s
Wall time: 5min 22s


In [40]:
# результат подбора гиперпараметров
log_study_als = study_als.trials_dataframe()
log_study_als.rename(
    columns={
        "values_0": "prec@10", "values_1": "recall@10", "values_2": "MAP@10", "values_3": "novelty",
        "values_4": "serendipity"
    },
    inplace=True
)
log_study_als

,number,prec@10,recall@10,MAP@10,novelty,serendipity,datetime_start,datetime_complete,duration,params_is_fitting_features,params_n_factors,user_attrs_is_fitting_features,user_attrs_n_factors,state
0,0,0.029617,0.146638,0.073554,4.853335,0.000026,2023-12-06 08:03:30.980864,2023-12-06 08:06:42.676477,0 days 00:03:11.695613,True,4,True,4.0,COMPLETE
1,1,0.031825,0.155255,0.075450,4.425382,0.000017,2023-12-06 08:06:42.677963,2023-12-06 08:08:53.531434,0 days 00:02:10.853471,False,4,False,4.0,COMPLETE
2,2,NaN,NaN,NaN,NaN,NaN,2023-12-06 08:08:53.534027,2023-12-06 08:08:53.534523,0 days 00:00:00.000496,False,4,NaN,NaN,PRUNED
3,3,NaN,NaN,NaN,NaN,NaN,2023-12-06 08:08:53.536716,2023-12-06 08:08:53.537156,0 days 00:00:00.000440,True,4,NaN,NaN,PRUNED


In [41]:
log_study_als.drop(log_study_als[log_study_als["state"] != "COMPLETE"].index, axis=0, inplace=True)
log_study_als

,number,prec@10,recall@10,MAP@10,novelty,serendipity,datetime_start,datetime_complete,duration,params_is_fitting_features,params_n_factors,user_attrs_is_fitting_features,user_attrs_n_factors,state
0,0,0.029617,0.146638,0.073554,4.853335,0.000026,2023-12-06 08:03:30.980864,2023-12-06 08:06:42.676477,0 days 00:03:11.695613,True,4,True,4.0,COMPLETE
1,1,0.031825,0.155255,0.075450,4.425382,0.000017,2023-12-06 08:06:42.677963,2023-12-06 08:08:53.531434,0 days 00:02:10.853471,False,4,False,4.0,COMPLETE


In [42]:
log_study_als["model"] = "ALS"
log_study_als.to_csv("log_study_als.csv", index=False)

In [43]:
%%time

study_lfm = optuna.create_study(
    study_name="LightFM",
    sampler=TPESampler(seed=42),
    directions=['maximize', 'maximize', 'maximize', 'maximize', 'maximize']
)
study_lfm.optimize(
    lambda trial: objective_lfm(
        trial=trial,
        config=CONFIG_MODELS,
        cv=cv,
        interactions_df=interactions,
        user_df=users,
        item_df=items,
        features=features,
        metrics=metrics
    ),
    n_trials=int(
        len(CONFIG_MODELS["N_FACTORS"]) *
        len(CONFIG_MODELS["USER_ALPHA"]) *
        len(CONFIG_MODELS["ITEM_ALPHA"]) *
        len(CONFIG_MODELS["lightfm_losses"]) *
        2
    )
)

[I 2023-12-06 08:08:53,647] A new study created in memory with name: LightFM


***RUN***model: LightFM
N_FACTORS: 4
loss: warp
user_alpha: 0
item_alpha: 0


==================== Fold 0

==================== Fold 1

==================== Fold 2


[I 2023-12-06 08:12:01,957] Trial 0 finished with values: [0.03390565902224099, 0.16586570746167056, 0.08094843885307158, 4.4341301996438505, 2.290517502310195e-05] and parameters: {'n_factors': 4, 'losses': 'warp', 'user_alpha': 0, 'item_alpha': 0}. 


***RUN***model: LightFM
N_FACTORS: 4
loss: bpr
user_alpha: 0
item_alpha: 0


==================== Fold 0

==================== Fold 1

==================== Fold 2


[I 2023-12-06 08:15:14,489] Trial 1 finished with values: [0.005007491647005787, 0.02709977383671658, 0.017615750809925636, 14.873565971222098, 2.571597910339995e-05] and parameters: {'n_factors': 4, 'losses': 'bpr', 'user_alpha': 0, 'item_alpha': 0}. 
[I 2023-12-06 08:15:14,492] Trial 2 pruned. 
[I 2023-12-06 08:15:14,496] Trial 3 pruned. 


CPU times: user 9min 4s, sys: 1min 1s, total: 10min 5s
Wall time: 6min 20s


In [44]:
log_study_lfm = study_lfm.trials_dataframe()
log_study_lfm.rename(
    columns={
        "values_0": "prec@10", "values_1": "recall@10", "values_2": "MAP@10", "values_3": "novelty",
        "values_4": "serendipity"
    },
    inplace=True
)
log_study_lfm.drop(log_study_lfm[log_study_lfm["state"] != "COMPLETE"].index, axis=0, inplace=True)
log_study_lfm["model"] = "LighFM"

In [45]:
log_study_lfm

,number,prec@10,recall@10,MAP@10,novelty,serendipity,datetime_start,datetime_complete,duration,params_item_alpha,params_losses,params_n_factors,params_user_alpha,user_attrs_losses,user_attrs_n_factors,user_attrs_user_alpha,state,model
0,0,0.033906,0.165866,0.080948,4.434130,0.000023,2023-12-06 08:08:53.652616,2023-12-06 08:12:01.956807,0 days 00:03:08.304191,0,warp,4,0,0.0,4.0,0.0,COMPLETE,LighFM
1,1,0.005007,0.027100,0.017616,14.873566,0.000026,2023-12-06 08:12:01.958443,2023-12-06 08:15:14.489640,0 days 00:03:12.531197,0,bpr,4,0,0.0,4.0,0.0,COMPLETE,LighFM


In [46]:
log_study_lfm.to_csv("log_study_lfm.csv", index=False)

Лучшие результаты обучения показывает модель lighFM с лосом warp

Теперь подберем для моделей с данным лоссом оптимальное user_alpha и item_alpha

In [47]:
CONFIG_MODELS_LFM = {
    "K_RECOS": 10,
    "RANDOM_STATE": 42,
    "NUM_THREADS": 16,
#     "N_FACTORS": (16, 32, ),
    "N_FACTORS": (4,),
    "N_EPOCHS": 10,  # Lightfm
    "USER_ALPHA": (0, 0.2),  # Lightfm
    "ITEM_ALPHA": (0, 0.2),  # Lightfm
    "LEARNING_RATE": 0.05,  # Lightfm
    "lightfm_losses": ('warp', ),  # Lightfm
}

In [48]:
%%time

study_lfm_alpha = optuna.create_study(
    study_name="LightFM_alpha",
    sampler=TPESampler(seed=42),
    directions=['maximize', 'maximize', 'maximize', 'maximize', 'maximize']
)
study_lfm_alpha.optimize(
    lambda trial: objective_lfm(
        trial=trial,
        config=CONFIG_MODELS_LFM,
        cv=cv,
        interactions_df=interactions,
        user_df=users,
        item_df=items,
        features=features,
        metrics=metrics
    ),
    n_trials=int(
        len(CONFIG_MODELS["N_FACTORS"]) *
        len(CONFIG_MODELS["USER_ALPHA"]) *
        len(CONFIG_MODELS["ITEM_ALPHA"]) *
        2
    )
)

[I 2023-12-06 08:15:14,558] A new study created in memory with name: LightFM_alpha


***RUN***model: LightFM
N_FACTORS: 4
loss: warp
user_alpha: 0.2
item_alpha: 0


==================== Fold 0

==================== Fold 1

==================== Fold 2


[I 2023-12-06 08:18:40,419] Trial 0 finished with values: [6.836791379823615e-05, 0.0002154310430293663, 6.498446907199415e-05, 15.91664118727129, 4.3724038184533286e-06] and parameters: {'n_factors': 4, 'losses': 'warp', 'user_alpha': 0.2, 'item_alpha': 0}. 


***RUN***model: LightFM
N_FACTORS: 4
loss: warp
user_alpha: 0
item_alpha: 0.2


==================== Fold 0

==================== Fold 1

==================== Fold 2


[I 2023-12-06 08:22:02,841] Trial 1 finished with values: [4.871105569308112e-05, 0.0001503337310009938, 4.039624653154397e-05, 15.914017012699157, 3.5642242700677773e-06] and parameters: {'n_factors': 4, 'losses': 'warp', 'user_alpha': 0, 'item_alpha': 0.2}. 


CPU times: user 9min 52s, sys: 1min 2s, total: 10min 54s
Wall time: 6min 48s


In [49]:
log_study_lfm_alpha = study_lfm_alpha.trials_dataframe()
log_study_lfm_alpha.rename(
    columns={
        "values_0": "prec@10", "values_1": "recall@10", "values_2": "MAP@10", "values_3": "novelty",
        "values_4": "serendipity"
    },
    inplace=True
)
log_study_lfm_alpha.drop(
    log_study_lfm_alpha[log_study_lfm_alpha["state"] != "COMPLETE"].index, axis=0, inplace=True
)
log_study_lfm_alpha["model"] = "LighFM"
log_study_lfm_alpha

,number,prec@10,recall@10,MAP@10,novelty,serendipity,datetime_start,datetime_complete,duration,params_item_alpha,params_losses,params_n_factors,params_user_alpha,user_attrs_losses,user_attrs_n_factors,user_attrs_user_alpha,state,model
0,0,0.000068,0.000215,0.000065,15.916641,0.000004,2023-12-06 08:15:14.560954,2023-12-06 08:18:40.419468,0 days 00:03:25.858514,0.0,warp,4,0.2,0.0,4,0.2,COMPLETE,LighFM
1,1,0.000049,0.000150,0.000040,15.914017,0.000004,2023-12-06 08:18:40.420921,2023-12-06 08:22:02.841791,0 days 00:03:22.420870,0.2,warp,4,0.0,0.2,4,0.0,COMPLETE,LighFM


In [50]:
log_study_lfm = pd.concat([log_study_lfm, log_study_lfm_alpha], ignore_index=True)
log_study_lfm

,number,prec@10,recall@10,MAP@10,novelty,serendipity,datetime_start,datetime_complete,duration,params_item_alpha,params_losses,params_n_factors,params_user_alpha,user_attrs_losses,user_attrs_n_factors,user_attrs_user_alpha,state,model
0,0,0.033906,0.165866,0.080948,4.434130,0.000023,2023-12-06 08:08:53.652616,2023-12-06 08:12:01.956807,0 days 00:03:08.304191,0.0,warp,4,0.0,0.0,4.0,0.0,COMPLETE,LighFM
1,1,0.005007,0.027100,0.017616,14.873566,0.000026,2023-12-06 08:12:01.958443,2023-12-06 08:15:14.489640,0 days 00:03:12.531197,0.0,bpr,4,0.0,0.0,4.0,0.0,COMPLETE,LighFM
2,0,0.000068,0.000215,0.000065,15.916641,0.000004,2023-12-06 08:15:14.560954,2023-12-06 08:18:40.419468,0 days 00:03:25.858514,0.0,warp,4,0.2,0.0,4.0,0.2,COMPLETE,LighFM
3,1,0.000049,0.000150,0.000040,15.914017,0.000004,2023-12-06 08:18:40.420921,2023-12-06 08:22:02.841791,0 days 00:03:22.420870,0.2,warp,4,0.0,0.2,4.0,0.0,COMPLETE,LighFM


In [51]:
log_study_lfm.to_csv("log_study_lfm.csv", index=False)

## Анализ метрик


In [52]:
log_study_als = pd.read_csv("log_study_als.csv")
log_study_lfm = pd.read_csv("log_study_lfm.csv")

По метрике **MAP@10** выберем по одной лучшей модели из ALS и LighFM:

ALS MAP@10
 - params_is_fitting_features = True
 - params_n_factors = 4

LighFM MAP@10
 - params_n_factors = 4
 - params_losses = warp
 - params_item_alpha = 0.2
 - params_user_alpha = 0.2

Но возьмем далее params_n_factors = 16


Обучение лучших моделей с валидации

Метод приближенного поиска соседей при помощи nmslib

In [54]:
interactions.drop(interactions.query("total_dur < 300").index, inplace=True)

In [55]:
 # Создание набора данных характеристик для пользователей
user_features = create_features_dataset(Columns.User, interactions, users, features["users"])
# Создание набора данных характеристик для айтема
item_features = create_features_dataset(Columns.Item, interactions, items, features["items"])

# Cоздаем датасет
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=features["users"],
    item_features_df=item_features,
    cat_item_features=features["items"]
        )

In [56]:
model_als = ImplicitALSWrapperModel(
    model=AlternatingLeastSquares(
        factors=16,
        random_state=42,
        num_threads=16,
    ),
    fit_features_together=True
)

In [57]:
model_als.fit(dataset)

/usr/local/lib/python3.10/dist-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


In [60]:
model_lfm = LightFMWrapperModel(
    LightFM(
        no_components=16,
        loss="warp",
        random_state=42,
        learning_rate=0.05,
        user_alpha=0.2,
        item_alpha=0.2,
    ),
    epochs=10,
    num_threads=16,
)

In [61]:
%%time

model_lfm.fit(dataset)

CPU times: user 2min 37s, sys: 242 ms, total: 2min 37s
Wall time: 1min 42s
